In [39]:
import csv
import escher
from escher import Builder
import pandas
from time import time, sleep

import cobra
from cobra.io import load_model, to_json, save_json_model
from cobra import Model, Reaction, Metabolite
from cobra.manipulation import delete_model_genes, knock_out_model_genes, remove_genes
from cobra.summary import Summary, MetaboliteSummary

model = load_model("iJN1463")

# The genes to be deleted one at a time
genes_to_delete = ["phaA","phaB","phaC2","benA","benB","benC","benD","phaG"]
gene_ids = ['PP_5003', 'PP_5004', 'PP_5005', 'PP_3161','PP_3162','PP_3163', 'PP_3164', 'PP_1408']

# Single deletion of each gene in my gene list
gene_list = gene_ids

remove_genes(model, gene_list)


In [40]:
# print(model.reactions)

# model.genes.get_by_id("PP_5003")

In [41]:
medium = model.medium
medium["EX_nh4_e"] = 4.5
medium["EX_ac_e"] = 25.0
medium["EX_glc__D_e"] = 0.0
model.medium = medium

# model.medium

# model.optimize()

phaA_reaction = Reaction("phaA")
phaA_reaction.name = 'acetyl-coa to acetoacetyl-coa'
phaA_reaction.subsystem = 'PHB Production'

phaA_reaction.add_metabolites({
    model.metabolites.get_by_id("accoa_c"): -2.0,
    model.metabolites.get_by_id("coa_c"): 1.0,
    model.metabolites.get_by_id("aacoa_c"): 1.0
})

# phaA_reaction.reaction

phaB_reaction = Reaction("phaB")
phaB_reaction.name = 'acetoacetyl-coa to 3-hydroxybutanoyl-coa'
phaB_reaction.subsystem = 'PHB Production'

phaB_reaction.add_metabolites({
    model.metabolites.get_by_id("nadph_c"): -1.0,
    model.metabolites.get_by_id("aacoa_c"): -1.0,
    model.metabolites.get_by_id("nadp_c"): 1.0,
    model.metabolites.get_by_id("3hbcoa_c"): 1.0
})

phaC_reaction = Reaction("PHAP2C4")
phaC_reaction.name = '3-hydroxybutanoyl-coa to PHB'
phaC_reaction.subsystem = 'PHB Production'

phaC_reaction.add_metabolites({
    model.metabolites.get_by_id("3hbcoa_c"): -1.0,
    model.metabolites.get_by_id("bhb_c"): -1.0,
    model.metabolites.get_by_id("PHAg_c"): 1.0,
    model.metabolites.get_by_id("coa_c"): 1.0
})


# print(phaC_reaction.reaction)

model.add_reactions([phaA_reaction, phaB_reaction, phaC_reaction])
# model.add_reactions([phaA_reaction, phaB_reaction])

model.objective = "PHAP2C4"
# model.objective = "phaB"
# model.objective = "BIOMASS_KT2440_WT3"

# biomass_rxn = model.reactions.get_by_id("BIOMASS_KT2440_WT3")

model.optimize()


,fluxes,reduced_costs
3HAD160,0.000000,0.000000e+00
13DAMPPabcpp,0.000000,-2.220446e-16
13DAMPPtex,0.000000,0.000000e+00
15DAPabcpp,0.000000,-1.110223e-16
1P2CBXLCYCL,0.000000,-8.800761e-17
...,...,...
4MCAT23DOX,0.000000,-4.857226e-17
4OD,0.000000,-6.245005e-17
phaA,0.000000,3.414107e-16
phaB,9.343158,1.387779e-16


In [42]:
# model.genes.get_by_id("PP_5005")

In [43]:
# from cobra.medium import minimal_medium

# max_growth = model.slim_optimize()
# minimal_medium(model, max_growth)

In [44]:
builder = Builder()

builder.model = model

# Run FBA with the model and add the flux data to the map
solution = builder.model.optimize()
builder.reaction_data = solution.fluxes

# Add some data for metabolites
builder.metabolite_data = solution.shadow_prices

# Simplify the map by hiding some labels
builder.hide_secondary_metabolites = True
builder.hide_all_labels = False

builder.reaction_scale = [
    { 'type': 'min', 'color': '#000000', 'size': 12 },
    { 'type': 'median', 'color': '#ffffff', 'size': 20 },
    { 'type': 'max', 'color': '#ff0000', 'size': 25 }
]

builder.reaction_scale_preset = 'GaBuRd'

# Make all the arrows three times as thick
builder.reaction_scale = [
    {k: v * 3 if k == 'size' else v for k, v in x.items()}
    for x in builder.reaction_scale
]

builder

Builder(hide_all_labels=False, hide_secondary_metabolites=True, metabolite_data={'10fthf_c': 0.0, '12dgr120_c'…